### Линейная регрессия на `pytorch`

В этом задании мы попробуем решить задачу линейно регрессии, где наша целевая переменная будет зависеть от двух признаков:

$$y = w_1 x_1 + w_2 x_2 + b$$

Мы сгенерируем датасет и попробуем на его основе узнать коэффициенты регрессии несколькими способами.

#### Математическая справка

***Дисклеймер***: *для решения задания нам потребуются только две формулы, выделенные цветом, справка дана исключительно для того, чтобы вы при желании могли разобраться, откуда эти формулы возникают.*

##### Переход к матричному виду

Значения коэффициентов линейной регрессии можно вычислить аналитически. Чтобы описать необходимое вычисление более компактно, давайте всё переведём в матричный вид, чтобы вычисление предсказаний модели выглядело так:
$$\mathbf{\hat{y}} = \mathbf{X} \mathbf{w}$$

где $\mathbf{\hat{y}}$ — это вектор со предсказаниями целевой переменной для всех наблюдений, $\mathbf{X}$ — матрица со значениями всех признаков для наших наблюдений, а $\mathbf{w}$ — вектор с параметрами линейной модели, его нам и нужно будет найти:

$$\mathbf{w} =
\begin{pmatrix}
b \\
w_1 \\
w_2
\end{pmatrix}
$$

Но в нашей модели три параметра, а признаков два! Поэтому, чтобы размерности параметров и признаков сошлись, мы ко всем наблюдениям добавим фиктивный признак, значение которого равно 1 — это стандартный трюк, позволяющий записывать линейные модели без явного свободного параметра $b$ в уравнениях.

Проверим, что мы ничего не сломали, и модель для всех наблюдений по-прежнему считается так, как задумано.

Вспоминаем, что при умножении матрицы $\mathbf{X}$ на вектор $\mathbf{w}$ мы получаем новый вектор, в $i$-м элементе которого будет результат скалярного произведения (т.е. суммы поэлементных произведений) $i$-й строки матрицы $\mathbf{X}$ на вектор $\mathbf{w}$:


$$
\mathbf{\hat{y}}[i] = \mathbf{X}[i] \cdot \mathbf{w} = (1, x_1^{(i)}, x_2^{(i)}) \cdot 
\begin{pmatrix}
b \\
w_1 \\
w_2
\end{pmatrix}
 = 1 \cdot b + x_1^{(i)} \cdot w_1 + x_2^{(i)} \cdot w_2
$$

То есть для всех наших наблюдений формула, получающая $\hat{y}$ по значениям параметров и признаков осталась той же, но у нас теперь есть удобная матричная запись.

$$
\color{violet}
\mathbf{\hat{y}} = \mathbf{X} \mathbf{w}
$$

##### Функция ошибки

Теперь мы хотим, чтобы наши предсказания были близки к верным значениям $\mathbf{y}$. Наше желание можно математически описать так: пусть суммарные квадраты ошибок предсказаний будут минимальны:

$$
\color{violet}
E(\mathbf{w}) = \sum_{i} (\mathbf{y}_i - \mathbf{\hat{y}}_i)^2 = \sum_{i} (\mathbf{y}_i - \mathbf{X}[i] \cdot \mathbf{w})^2  \rightarrow \min
$$

В матричной записи это будет выглядеть так:

$$
\color{violet}
E(\mathbf{w}) = (\mathbf{y} - \mathbf{X} \mathbf{w})^T (\mathbf{y} - \mathbf{X} \mathbf{w}) \rightarrow \min
$$

**Поясним за нотацию**: *$(\mathbf{y} - \mathbf{X} \mathbf{w})^T$ означает, что мы транспонируем вектор $(\mathbf{y} - \mathbf{X} \mathbf{w})$, то есть рассматриваем его как вектор-строку, а не вектор-столбец.*


То есть мы считаем ошибки для всех наблюдений, складываем их в вектор, и считаем его скалярное произведение с самим собой.



##### Градиент и аналитическое решение

Чтобы найти локальный минимум этой функции, мы можем посчитать градиент нашей функции ошибки по параметрам:

$$
\frac{\partial E(\mathbf{w})}{\partial \mathbf{w}}
= -2 \mathbf{X}^T (\mathbf{y} - \mathbf{X} \mathbf{w})
$$

Это нам пригодится, когда мы будем искать решение градиентным спуском.

***Замечание***: *двойка перед скобкой не влияет на направление градиента, и для удобства её часто опускают. Чтобы не терять математической строгости, именно поэтому функция ошибки часто включает множитель $\frac{1}{2}$, чтобы потом при взятии производных $2$ и $\frac{1}{2}$ взаимно сократились:*


$$
E(\mathbf{w}) = \frac{1}{2} (\mathbf{y} - \mathbf{X} \mathbf{w})^T (\mathbf{y} - \mathbf{X} \mathbf{w}) \rightarrow \min
$$

$$
\color{green}
\frac{\partial E(\mathbf{w})}{\partial \mathbf{w}} = \nabla_\mathbf{w} E(\mathbf{w})
= - \mathbf{X}^T (\mathbf{y} - \mathbf{X} \mathbf{w})
$$


Но оказывается, что для нашей модели значение параметров $\mathbf{w}$, которые минимизируют эту ошибку, можно найти за один шаг!
У нашей функции ошибки единственный локальный минимум (значит, он же и глобальный), а в нём градиент должен быть равен нулю. Значит нужно решить уравнение:
$$
% \begin{align*}
\nabla_\mathbf{w} E(\mathbf{w})
= -\mathbf{X}^T (\mathbf{y} - \mathbf{X} \mathbf{w}) = \mathbf{0}
$$

$$
\mathbf{X}^T \mathbf{X} \mathbf{w} = \mathbf{X}^T \mathbf{y} \\
$$
$$
(\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{X} \mathbf{w} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}
$$


$$
\color{red}
\mathbf{w} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}
$$

**Поясним за нотацию**: *$(\mathbf{X}^T \mathbf{X})^{-1}$ — это обратная матрица от матричного произведения $\mathbf{X}^T$ на $\mathbf{X}$*

#### Смоделируем наши данные

In [1]:
import torch

In [2]:
# фиксируем генератор случайных чисел
torch.manual_seed(42)

n_observations = 64

x1 = torch.randn(n_observations)
x2 = torch.randn(n_observations)

Смоделируем $y = 2x_1 - 3x_2 + 1 + \varepsilon$, где $\varepsilon \sim \mathcal{N}(0, 1)$ — небольшой шум

То есть наши истинные значения параметров:

$$\mathbf{w} =
\begin{pmatrix}
b \\
w_1 \\
w_2
\end{pmatrix}
 =
\begin{pmatrix}
\phantom{-}1 \\
\phantom{-}2 \\
-3
\end{pmatrix}
$$

In [3]:
eps = torch.randn(n_observations)

y = 2*x1 - 3*x2 + 1 + eps

Можем посмотреть на наши данные:

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter3d(
    x=x1,
    y=x2,
    z=y,
    mode='markers',
    marker=dict(
        size=12,
        color=y,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ),
    hovertemplate="x1=%{x:.2f}, x2=%{y:.2f}, y=%{z:.2f}",
)])

# tight layout
fig.update_layout(
    scene=dict(
        xaxis_title="x1",
        yaxis_title="x2",
        zaxis_title="y",
        aspectratio=dict(x=1, y=1, z=1)
    )
)
fig.show()

#### Задание 1. Аналитическое решение

Давайте посчитаем точное решение с помощью Pytorch

$$
\color{red}
\mathbf{w} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}
$$

Вам понадобится:
1. Функция, которая сможет склеивать векторы отдельных признаков в единую матрицу. В этот раз нам подойдёт функция `torch.stack`, но посмотрите также на функцию `torch.cat`, они делают похожие вещи, но немного по-разному.
2. Операция матричного умножения — это функция `torch.matmul` или оператор `@`, его мы уже видели
3. Операция транспонирования $X^T$ — через атрибут `T` тензора.
4. Функция для расчёта обратной матрицы — `torch.linalg.inv`

In [ ]:
dummy_feature = torch.ones_like(x1)

X = torch.stack(...)  # объедините dummy_feature, x1 и x2 в один тензор признаков, его форма: [64, 3]

w_exact = ...  # посчитайте параметры модели по красной формуле
w_exact

#### Задание 2. Градиентный спуск вручную

Теперь будем решать задачу градиентным спуском — это будет наш главный способ обучать более сложные модели.

На каждой итерации мы будем:
1. Оценивать градиент $\color{green} \nabla_\mathbf{w} E(\mathbf{w}) = -\mathbf{X}^T (\mathbf{y} - \mathbf{X} \mathbf{w})$
2. Шагать в противоположную сторону с небольшим шагом $\eta$: $\mathbf{w}^{(t+1)} = \mathbf{w}^{(t)} - \eta \cdot \nabla_\mathbf{w} E(\mathbf{w})$

Нам потребуется определить:
- начальное значение $\mathbf{w}$ — выберем из стандартного нормального распределения
- размер шага обновления $\eta$
- количество шагов

In [ ]:
w = torch.randn(3)  # начальное значение
learning_rate = 0.005  # шаг оптимизации
n_steps = 100  # число итераций
w

In [ ]:
for i in range(n_steps):
    grad = ...  # посчитайте по зелёной формуле
    w -= learning_rate * grad
    if (i+1) % 10 == 0:
        print(f"step {i+1}: w = {w}, gradient = {grad}")

#### Задание 3. Автоматический градиентный спуск

Эту сторону pytorch мы с вами ещё подробно не смотрели, но давайте немного забежим вперёд и просто посмотрим, чем он нам так полезен.

Выше в ручном градиентном спуске при расчёте градиентов мы пользовались готовой формулой для линейной регрессии, которую старательно выводили в математической справке.

Но мы точно не хотели бы делать что-то подобное для глубоких нейронных сетей: выкладки были бы очень громоздкими.
В лекции мы говорили, что pytorch умеет считать градиенты самостоятельно. Давайте посмотрим, как мы будем этим пользоваться.

**Первое отличие**: тензор с нашими параметрами мы помечаем как `requires_grad=True`; так pytorch будет знать, что этот тензор требует расчёта градиентов.

In [ ]:
w = torch.randn(3, requires_grad=True)  # начальное значение
learning_rate = 0.005  # шаг оптимизации
n_steps = 100  # число итераций
w

**Второе отличие**: теперь вместо явной формулы для расчёта градиентов нам просто нужно получить вычисленное значение ошибки при текущих значениях параметров, которое мы сохраним в переменную `loss`. Затем мы вызовем у неё метод `.backward`, чтобы рассчитались градиенты для всех тензоров, которые участвуют в вычислении ошибки и помечены как `requires_grad=True`; в нашем случае это только вектор параметров `w`.

Посчитанные градиенты попадут в атрибут `grad` нашего тензора, и далее мы используем их, чтобы обновить значения параметров и перейти к следующей итерации.

Вам в этом задании нужно сделать две вещи:
- Получить предсказания модели `y_hat` из признаков `X` и текущих значений параметров `w`: $$\color{violet}\mathbf{\hat{y}} = \mathbf{X} \mathbf{w}$$
- Рассчитать значение ошибки из предсказаний `y_hat` и целевой переменной `y`: $$\color{violet} E(\mathbf{w}) = \sum_{i} (\mathbf{y}_i - \mathbf{\hat{y}}_i)^2$$

In [ ]:
for i in range(n_steps):
    # прямой проход
    y_hat = ...  # предсказания модели на основе X и w
    loss = ...  # значение ошибки

    # обратный проход
    loss.backward()  # рассчитываем градиенты

    # обновляем параметры, локально отключая расчёт градиентов, чтобы не запутать pytorch
    with torch.no_grad():
        w -= learning_rate * w.grad
        if (i+1) % 10 == 0:
            print(f"step {i+1}: w = {w}, gradient = {w.grad}")

    # обнуляем градиенты, чтобы они не суммировались накопленным итогом
    w.grad = None